In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt   
import re 
import io
import ipywidgets as widgets

from typing import Optional
from datetime import date
from IPython.display import display
 
from src.plot_parameters import ArrayValue, ArrayType
from src.utils_v3 import LinePlotParameters, StackPlotParameters, StyleParameters
from src.utils_v3 import make_lineplot, make_comparative_lineplots

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
def fix_col_names(df:pd.DataFrame) -> None:
    col_names_to_change = df.columns[df.columns.str.startswith("=")].tolist()
    changed_col_names = [col.replace("=", "") for col in col_names_to_change]
    changed_col_names = [col.replace("\"", "") for col in changed_col_names]
    df.rename(columns=dict(zip(col_names_to_change, changed_col_names)), inplace=True)

In [4]:
def generate_names_from_dataframe(
                        input_df: pd.DataFrame, 
                        variable_dict: dict, 
                        array_val_dict: dict):
    
    rootname_pattern = re.compile(r"([\w\s$%/-]+):\s([\w\s$%/-]+)\.([\w\s$%/-]+)\[?") 
    array_pattern = re.compile(r"\[(\w+|\w+(,\s\w+){0,3})\]")
    stella_run_names = set()
    for col in input_df.columns:
        for match in rootname_pattern.findall(col):
            stella_run_names.add(match[0])
            variable_name = f"{match[1]}.{match[2]}"
            
            if match[1] not in variable_dict.keys():
                variable_dict[match[1]] = set([match[2]])
            else:
                variable_dict[match[1]].add(match[2])
        for match in array_pattern.findall(col):
            if variable_name not in array_val_dict.keys():
                array_val_dict[variable_name] = [ArrayValue(array_val.strip()) \
                        for array_val in match[0].split(",") if array_val != ""]
    return list(stella_run_names)

# Widgets

In [5]:
LIBRA_outputs_upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False
)

In [6]:
stella_runname_dropdown = widgets.SelectMultiple(
    options=[None],
    value=[None],
    description='LIBRA run names:',
    style={'description_width': 'initial'},
    disabled=False
) 

In [7]:
module_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Name of LIBRA module:',
    style={'description_width': 'initial'},
    disabled=False,
)

In [8]:
variablename_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Name of LIBRA variable:',
    style={'description_width': 'initial'},
    disabled=False,
)

In [9]:
arrayval_dropdown_1 = widgets.Dropdown(
    options=[],
    value=None,
    description='Array values 1:',
    style={'description_width': 'initial'},
    disabled=False,
)
arrayval_dropdown_2 = widgets.Dropdown(
    options=[],
    value=None,
    description='Array values 2:',
    style={'description_width': 'initial'},
    disabled=False,
)
arrayval_dropdown_3 = widgets.Dropdown(
    options=[],
    value=None,
    description='Array values 3:',
    style={'description_width': 'initial'},
    disabled=False,
)

In [10]:
title_box = widgets.Text(value="Plot title",
                        placeholder="Enter plot title.",
                        description="Lineplot title:",
                        style={'description_width': 'initial'},
                        disabled=False)

In [11]:
ylabel_box = widgets.Text(value="Y-axis label",
                        placeholder="Enter y-axis label.",
                        description="Y-axis label:",
                        style={'description_width': 'initial'},
                        disabled=False)

In [12]:
max_yval_select = widgets.BoundedFloatText(value=0.0,
                                        min=0.0,
                                        max=10000000.0,
                                        step=0.1,
                                        description='Maximum value for y-axis:',
                                        style={'description_width': 'initial'},
                                        disabled=False)

In [13]:
isdecimal_checkbox = widgets.Checkbox(
    value=False,
    description='Decimal point for y-axis values',
    disabled=False,
    indent=False
)

In [14]:
isexogenousinput_checkbox = widgets.Checkbox(
    value=False,
    description='Exogenous input',
    disabled=False,
    indent=False
)

In [15]:
errormessage_box = widgets.HTML(
    value="",
    placeholder='No errors.',
    description='',
)

# Function to update plot

In [16]:
def update_plot(
    input_file: tuple,
    run_names: list[str], 
    module: str, 
    variable: str, 
    array_val_1: str,
    array_val_2: str,
    array_val_3: str,
    title: str,
    y_label: Optional[str],
    max_yval: Optional[float],
    decimal: Optional[bool],
    is_exogenous_input: Optional[bool]
) -> None:
      
    try:
        uploaded_file = input_file[0]
        input_df = pd.read_csv(io.BytesIO(uploaded_file.content), index_col=0).astype(np.float32)
        fix_col_names(input_df)
    except IndexError as e:
        errormessage_box.value = "Please upload a LIBRA output csv file."
        return    
    try:    
        style_params = StyleParameters(
                                stella_run_names=run_names,
                                compare=False
                            )
    except AssertionError as e:
        errormessage_box.value = str(e)
        return       
    try:
        plot_params = LinePlotParameters(
                    module=module,
                    variable=variable,
                    array_vals=[array_val for array_val in \
                                [array_val_1, array_val_2, array_val_3] if array_val],
                    title=title,
                    y_label=y_label,
                    max_yval=max_yval if max_yval > 0 else None,
                    decimal=decimal,
                    is_exogenous_input=is_exogenous_input
            )
    except AssertionError as e:
        errormessage_box.value = str(e)
        return
    
    try:
        make_lineplot(input_df, plot_params, style_params)
        make_comparative_lineplots(input_df, plot_params, style_params)
        errormessage_box.value = ""
    except KeyError as k:
        errormessage_box.value = f"{str(k)} is not in the uploaded csv file."
        plt.close()
        return
        

# Functions to update widgets

In [17]:
def update_multiselect(multiselect, options_list: list[str]) -> None:
    multiselect.options = sorted(options_list)
    multiselect.value = [multiselect.options[0]]

def update_dropdown(dropdown, options_list: list[str]) -> None:
    dropdown.options = sorted(options_list)
    dropdown.value = dropdown.options[0]

In [18]:
array_val_dict = {}
variable_dict = {}
def post_upload(*args):
    uploaded_file = LIBRA_outputs_upload_widget.value[0]
    input_df = pd.read_csv(io.BytesIO(uploaded_file.content), index_col=0).astype(np.float32)
    fix_col_names(input_df)
    
    array_val_dict.clear()
    variable_dict.clear()
    stella_run_names = generate_names_from_dataframe(input_df, variable_dict, array_val_dict)
    update_multiselect(stella_runname_dropdown, stella_run_names)
    update_dropdown(module_dropdown, list(variable_dict.keys()))

In [19]:
def update_variablename_dropdown(*args) -> None:
    variable_list = list(variable_dict[module_dropdown.value])
    update_dropdown(variablename_dropdown, variable_list)

In [20]:
def update_arrayval_dropdowns(*args) -> None:
    try:
        arrayvals = array_val_dict[f"{module_dropdown.value}.{variablename_dropdown.value}"]
    except KeyError as e:
        errormessage_box.value = f"{str(e)} is an invalid combination of a module and a variable name."
        return
    for i, arrayval in enumerate(arrayvals):
        update_dropdown(arrayval_dropdowns[i], ArrayType.enumerate_array_type(arrayval.data_type))
    for arrayval_dropdown in arrayval_dropdowns[len(arrayvals):]:
        arrayval_dropdown.options = []
        arrayval_dropdown.value = None
        

# UI elements

In [21]:
out = widgets.interactive_output(
    update_plot,    
    {
        'input_file': LIBRA_outputs_upload_widget,
        'run_names': stella_runname_dropdown,
        'module': module_dropdown,
        'variable': variablename_dropdown, 
        'array_val_1': arrayval_dropdown_1,
        'array_val_2': arrayval_dropdown_2,
        'array_val_3': arrayval_dropdown_3,
        'title': title_box,
        'y_label': ylabel_box,
        'max_yval': max_yval_select,
        'decimal': isdecimal_checkbox,
        'is_exogenous_input': isexogenousinput_checkbox
    }
)
arrayval_dropdowns = [arrayval_dropdown_1, arrayval_dropdown_2, arrayval_dropdown_3]

In [22]:
variable_params = widgets.HBox([stella_runname_dropdown, module_dropdown, variablename_dropdown])
plot_element_params = widgets.HBox([title_box, ylabel_box, max_yval_select])
array_params = widgets.HBox(arrayval_dropdowns)
checkbox_params = widgets.HBox([isdecimal_checkbox, isexogenousinput_checkbox])

# Callbacks

In [23]:
LIBRA_outputs_upload_widget.observe(post_upload, names="value")
module_dropdown.observe(update_arrayval_dropdowns, names="value")
variablename_dropdown.observe(update_arrayval_dropdowns, names="value")
module_dropdown.observe(update_variablename_dropdown, names="value")

# Main dashboard

In [24]:
display(errormessage_box,
        LIBRA_outputs_upload_widget,  
        variable_params, 
        array_params, 
        plot_element_params, 
        checkbox_params, 
        out)

HTML(value='Please upload a LIBRA output csv file.', placeholder='No errors.')

FileUpload(value=(), accept='.csv', description='Upload')

Output()

In [25]:
%load_ext watermark

In [26]:
# matplotlib, ipywidgets, numpy, pandas, IPython
%watermark -v -m -p ipywidgets,numpy,pandas,matplotlib

%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

ipywidgets: 8.0.1
numpy     : 1.23.2
pandas    : 1.4.3
matplotlib: 3.5.3

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : 
CPU cores   : 8
Architecture: 64bit

Last updated: Sun Aug 28 2022 08:27:57MDT

